## EX 3: Plot classification probability

## (一)資料匯入及描述
* 首先先匯入iris 鳶尾花資料集，使用`iris = datasets.load_iris()`將資料存入
* 準備X (特徵資料) 以及 y (目標資料)，僅使用兩個特徵方便視覺呈現

In [10]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data[:, 0:2]  # 僅使用前兩個特徵，方便視覺化呈現
y = iris.target

n_features = X.shape[1]

* `iris`為一個dict型別資料，我們可以用以下指令來看一下資料的內容。

In [ ]:
for key,value in iris.items() :
    try:
        print (key,value.shape)
    except:
        print (key)

| 顯示 | 說明 |
| -- | -- |
| ('target_names', (3L,))| 共有三種鳶尾花 setosa, versicolor, virginica |
| ('data', (150L, 4L)) | 有150筆資料，共四種特徵 |
| ('target', (150L,))| 這150筆資料各是那一種鳶尾花|
| DESCR | 資料之描述 |
| feature_names| 四個特徵代表的意義 |

## (二) 分類器的選擇
這個範例選擇了四種分類器，存入一個dict資料中，分別為：
1. L1 logistic
2. L2 logistic (OvR)
3. Linear SVC
4. L2 logistic (Multinomial)

其中`LogisticRegression` 並不適合拿來做多目標的分類器，我們可以用結果圖的分類機率來觀察。

In [ ]:
C = 1.0

# Create different classifiers. The logistic regression cannot do
# multiclass out of the box.
classifiers = {'L1 logistic': LogisticRegression(C=C, penalty='l1'),
               'L2 logistic (OvR)': LogisticRegression(C=C, penalty='l2'),
               'Linear SVC': SVC(kernel='linear', C=C, probability=True,
                                 random_state=0),
               'L2 logistic (Multinomial)': LogisticRegression(
                C=C, solver='lbfgs', multi_class='multinomial'
                )}

n_classifiers = len(classifiers)

`np.linspace(起始, 終止, 數量)` 目的為產生等間隔之數據，例如`print(np.linspace(1,3,3))` 的結果為 `[ 1.  2.  3.]`，而`print(np.linspace(1,3,5))`的結果為 `[ 1.   1.5  2.   2.5  3. ]`，而`np.meshgrid(xx,yy)`則用來產生網格狀座標。而`numpy.c_` 為numpy特殊物件，能協助將numpy 陣列連接起來，以下範例可以展示相關函式用法。
```python
xx, yy = np.meshgrid(np.linspace(1,3,3), np.linspace(1,3,3))
print('xx= \n%s' % xx)
print('yy= \n%s' % yy)
print('xx.ravel()= \n%s' % xx.ravel())
print('np.c_[xx.ravel(), yy.ravel()]= \n%s' % np.c_[xx.ravel(), yy.ravel()])
```
結果顯示為
```
xx= 
[[ 1.  2.  3.]
 [ 1.  2.  3.]
 [ 1.  2.  3.]]
 
yy= 
[[ 1.  1.  1.]
 [ 2.  2.  2.]
 [ 3.  3.  3.]]
 
xx.ravel()= 
[ 1.  2.  3.  1.  2.  3.  1.  2.  3.]
```

其中numpy.c_ 為numpy特殊物件，能協助將numpy 陣列連接起來，使用方法如下
```python
Xfull = np.c_[xx.ravel(), yy.ravel()]
>>> array([[1, 2, 3, 0, 0, 4, 5, 6]])
```


In [9]:
plt.figure(figsize=(3 * 2, n_classifiers * 2))
plt.subplots_adjust(bottom=.2, top=.95)

xx = np.linspace(3, 9, 100)
yy = np.linspace(1, 5, 100).T
xx, yy = np.meshgrid(xx, yy)
Xfull = np.c_[xx.ravel(), yy.ravel()]

for index, (name, classifier) in enumerate(classifiers.items()):
    classifier.fit(X, y)

    y_pred = classifier.predict(X)
    classif_rate = np.mean(y_pred.ravel() == y.ravel()) * 100
    print("classif_rate for %s : %f " % (name, classif_rate))

    # View probabilities=
    probas = classifier.predict_proba(Xfull)
    n_classes = np.unique(y_pred).size
    for k in range(n_classes):
        plt.subplot(n_classifiers, n_classes, index * n_classes + k + 1)
        plt.title("Class %d" % k)
        if k == 0:
            plt.ylabel(name)
        imshow_handle = plt.imshow(probas[:, k].reshape((100, 100)),
                                   extent=(3, 9, 1, 5), origin='lower')
        plt.xticks(())
        plt.yticks(())
        idx = (y_pred == k)
        if idx.any():
            plt.scatter(X[idx, 0], X[idx, 1], marker='o', c='k')

ax = plt.axes([0.15, 0.04, 0.7, 0.05])
plt.title("Probability")
plt.colorbar(imshow_handle, cax=ax, orientation='horizontal')

plt.show()

classif_rate for L2 logistic (OvR) : 76.666667 
classif_rate for L1 logistic : 79.333333 
classif_rate for Linear SVC : 82.000000 
classif_rate for L2 logistic (Multinomial) : 82.000000 


In [7]:
import numpy as np
xx = np.linspace(3, 9, 100)
yy = np.linspace(1, 5, 100).T
xx, yy = np.meshgrid(xx, yy)
print(yy)

[[ 1.          1.          1.         ...,  1.          1.          1.        ]
 [ 1.04040404  1.04040404  1.04040404 ...,  1.04040404  1.04040404
   1.04040404]
 [ 1.08080808  1.08080808  1.08080808 ...,  1.08080808  1.08080808
   1.08080808]
 ..., 
 [ 4.91919192  4.91919192  4.91919192 ...,  4.91919192  4.91919192
   4.91919192]
 [ 4.95959596  4.95959596  4.95959596 ...,  4.95959596  4.95959596
   4.95959596]
 [ 5.          5.          5.         ...,  5.          5.          5.        ]]


In [11]:
xx, yy = np.meshgrid(np.linspace(1,3,3), np.linspace(1,3,3))

In [19]:
print(np.linspace(1,3,5))

[ 1.   1.5  2.   2.5  3. ]


In [26]:
xx, yy = np.meshgrid(np.linspace(1,3,3), np.linspace(1,3,3))
print('xx= \n%s' % xx)
print('yy= \n%s' % yy)
print('xx.ravel()= \n%s' % xx.ravel())

xx= 
[[ 1.  2.  3.]
 [ 1.  2.  3.]
 [ 1.  2.  3.]]
yy= 
[[ 1.  1.  1.]
 [ 2.  2.  2.]
 [ 3.  3.  3.]]
xx.ravel()= 
[ 1.  2.  3.  1.  2.  3.  1.  2.  3.]


In [25]:
xx.ravel()

array([ 1.,  2.,  3.,  1.,  2.,  3.,  1.,  2.,  3.])

In [27]:
xx, yy = np.meshgrid(np.linspace(1,3,3), np.linspace(1,3,3))
print('xx= \n%s' % xx)
print('yy= \n%s' % yy)
print('xx.ravel()= \n%s' % xx.ravel())
print('np.c_[xx.ravel(), yy.ravel()]= \n%s' % np.c_[xx.ravel(), yy.ravel()])

xx= 
[[ 1.  2.  3.]
 [ 1.  2.  3.]
 [ 1.  2.  3.]]
yy= 
[[ 1.  1.  1.]
 [ 2.  2.  2.]
 [ 3.  3.  3.]]
xx.ravel()= 
[ 1.  2.  3.  1.  2.  3.  1.  2.  3.]
np.c_[xx.ravel(), yy.ravel()]= 
[[ 1.  1.]
 [ 2.  1.]
 [ 3.  1.]
 [ 1.  2.]
 [ 2.  2.]
 [ 3.  2.]
 [ 1.  3.]
 [ 2.  3.]
 [ 3.  3.]]
